In [1]:
import pandas as pd
import numpy as np
import janitor
import pickle
import os
import operator
import functools
import itertools
from typing import Callable

import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
if 'raw_databases.pkl' not in os.listdir('./data/'):
    from quickstart.loader import XlsxDriveLoader

    Loads = XlsxDriveLoader()  # Drive Folder is hardcoded in module, since this is not prone to change
    databases = Loads.content  # Process takes approximately ~3 minutes to run for the first time.
                               # Then the file will be stored in the data/
else:
    with open('./data/raw_databases.pkl', 'rb') as file:
        databases = pickle.load(file)

In [3]:
(db_name_contaminantes, db_name_meteoorologicas, 
db_namea_notaciones, db_name_estaciones) = databases.keys()


get_col_names: Callable[[pd.core.frame.DataFrame], pd.core.indexes.base.Index] = (
                         lambda df: sorted(list(df.columns)))

def shared_structure(db_name: str) -> list:
    """Returns sheets in Dict of DataFerames that conain the same fields"""

    df_col_names = [[key, get_col_names(val)] for key, val in databases[db_name].items()]

    grouped = list()
    for _, g in itertools.groupby(df_col_names, operator.itemgetter(1)):
        group = list(g) 
        if len(group) > 1: # Can't be performed with comprehension since g is a generator
            grouped.append(group)
    
    if len(grouped) == 1: 
        return [sheet_name for sheet_name, _ in functools.reduce(operator.iconcat, grouped, [])] 
        # Expecting there is only one shared_strcuture across the DataFrames
    else:
        return ... # Not necessary

In [4]:
contaminantes_ = pd.concat([databases[db_name_contaminantes][sheet] 
                            for sheet in shared_structure(db_name_contaminantes)])

meteorologicas_ = pd.concat([databases[db_name_meteoorologicas][sheet] 
                            for sheet in shared_structure(db_name_meteoorologicas)])

estaciones_ = pd.concat([databases[db_name_estaciones][sheet] 
                            for sheet in shared_structure(db_name_estaciones)])

In [5]:
fields = ['parametro', 'Fecha',
          'SO2', 'SO2 b'] # San Pedro Ids

def munge(df: pd.core.frame.DataFrame, fields: list):
    return (
        df
        .loc[:, fields]
        .pivot(index='Fecha', columns='parametro', values=['SO2', 'SO2 b'])
        # Apply dictionary into df
        .pipe(janitor.clean_names, strip_underscores='r')
        # Fix dtypes
    )

In [6]:
contaminantes = munge(contaminantes_, fields)
meteorologicas = munge(meteorologicas_, fields)

In [7]:
# There is a Timestamp missing in df metereologicas
set(contaminantes.index).difference(set(meteorologicas.index))

{Timestamp('2019-05-15 20:00:00')}

In [8]:
df = contaminantes.merge(meteorologicas,
                         left_index=True, right_index=True,
                         how='outer')

In [9]:
df

so2                                                     \
parametro              co   no    no  no2   no2  nox   nox   o3    o3   pm10   
Fecha                                                                          
2017-01-01 00:00:00   1.1  NaN  17.0  NaN  20.1  NaN  37.0  NaN   6.0  163.0   
2017-01-01 01:00:00  1.05  NaN  19.5  NaN  25.1  NaN  44.5  NaN   5.0  117.0   
2017-01-01 02:00:00  0.91  NaN  16.7  NaN  22.5  NaN  39.0  NaN   5.0   46.0   
2017-01-01 03:00:00  0.75  NaN  12.5  NaN  16.8  NaN  29.2  NaN   8.0   36.0   
2017-01-01 04:00:00  0.59  NaN   6.5  NaN   6.7  NaN  13.1  NaN  12.0   25.0   
...                   ...  ...   ...  ...   ...  ...   ...  ...   ...    ...   
2021-06-30 19:00:00  0.67  4.4   NaN  NaN  11.3  NaN  15.7  NaN  17.0   36.0   
2021-06-30 20:00:00  0.66  NaN   3.9  NaN  10.7  NaN  14.6  NaN  16.0   41.0   
2021-06-30 21:00:00  0.64  NaN   3.4  NaN  11.7  NaN  15.1  NaN  15.0   16.0   
2021-06-30 22:00:00  0.61  NaN   3.4  NaN   9.7  NaN  13.1  NaN  16.0   31.0   
2021-06-30 23:00:00   0.6  NaN   3.5  NaN  12.0  NaN  15.5  NaN  14.0   37.0   

                     ... so2_b                                                
parametro            ...   prs rainf   rh   rh   sr tout  wdr  wdr  wsr  wsr  
Fecha                ...                                                      
2017-01-01 00:00:00  ...   NaN   NaN  NaN  NaN    l  NaN  NaN  NaN  NaN  NaN  
2017-01-01 01:00:00  ...   NaN   NaN  NaN  NaN    l  NaN  NaN  NaN  NaN  NaN  
2017-01-01 02:00:00  ...   NaN   NaN  NaN  NaN    l  NaN  NaN  NaN  NaN  NaN  
2017-01-01 03:00:00  ...   NaN   NaN  NaN  NaN    l  NaN  NaN  NaN  NaN  NaN  
2017-01-01 04:00:00  ...   NaN   NaN  NaN  NaN    l  NaN  NaN  NaN  NaN  NaN  
...                  ...   ...   ...  ...  ...  ...  ...  ...  ...  ...  ...  
2021-06-30 19:00:00  ...   NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
2021-06-30 20:00:00  ...   NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
2021-06-30 21:00:00  ...   NaN   NaN  NaN  NaN    l  NaN  NaN  NaN  NaN  NaN  
2021-06-30 22:00:00  ...   NaN   NaN  NaN  NaN    l  NaN  NaN  NaN  NaN  NaN  
2021-06-30 23:00:00  ...   NaN   NaN  NaN  NaN    l  NaN  NaN  NaN  NaN  NaN  

[39394 rows x 46 columns]

In [10]:
with open('data/dataframe.pkl', 'wb') as file:
            pickle.dump(df, file)